In [17]:
import requests
import pandas as pd
%pip install altair pandas vega_datasets
%pip install -U altair_viewer
import altair as alt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### creating data frame from US census data on populations of states in 2019

In [18]:
jsonlist = requests.get("https://api.census.gov/data/2019/pep/charagegroups?get=NAME,POP&for=state:*").json()
us_states = []
us_state_populations = []

for list in jsonlist:
    us_states.append(list[0])
    us_state_populations.append(list[1])
us_state_population_data = {"State":us_states,"Population":us_state_populations}
us_population_df = pd.DataFrame(us_state_population_data)
us_population_df = us_population_df.iloc[1:]
us_population_df["Population"] = pd.to_numeric(us_population_df["Population"])
us_population_df.head()





,State,Population
1,Mississippi,2976149
2,Missouri,6137428
3,Montana,1068778
4,Nebraska,1934408
5,Nevada,3080156


Vega-altair uses its own id system that I could not find anywhere else so I used one of their datasets to retrieve their ids

In [19]:
from vega_datasets import data
# values in vega dataset stored alphabetically so dataframe was sorted alphabetically
us_population_df=us_population_df.sort_values(by="State")
us_population_df=us_population_df.reset_index(drop=True)

altair_state_ids = data.population_engineers_hurricanes()

us_population_df["id"]=altair_state_ids['id']
us_population_df.head()


,State,Population,id
0,Alabama,4903185,1
1,Alaska,731545,2
2,Arizona,7278717,4
3,Arkansas,3017804,5
4,California,39512223,6


In [20]:
from vega_datasets import data

states_geo = alt.topo_feature(data.us_10m.url, 'states')


alt.Chart(states_geo).mark_geoshape().encode(
    color='Population:Q',
    tooltip=['State:N', 'Population:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(us_population_df, 'id', ['Population','State'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)


/Users/maddonk/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/maddonk/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/maddonk/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

Some data was not as easily available from the us census api so we decided to scrape their online data tables instead

In order to do this we needed the state codes as this is what the website used in their url to indicate different states

In [21]:
state_codes = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]
state_names = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California",
    "Colorado", "Connecticut", "Delaware", "Florida", "Georgia",
    "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas",
    "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts",
    "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana",
    "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico",
    "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma",
    "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota",
    "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington",
    "West Virginia", "Wisconsin", "Wyoming"
]

Here each state on us census 'quickfacts' is cycled through to collect the average household median income between 2017 and 2021 (2021 dollars)

In [22]:
from scrapy import Selector
state_median_incomes =[]
for state in state_codes:
   
    quickfacts_url= 'https://www.census.gov/quickfacts/fact/table/'+state+'/INC110221#INC110221'
   
    response = requests.get(quickfacts_url)
    sel= Selector(text=response.text)
    state_median_incomes.append(sel.css('td::text').getall()[3])
    print(state)

state_household_income_df = pd.DataFrame({"State":state_names,"Median Household income":state_median_incomes})
state_household_income_df.head()

AL
AK
AZ
AR
CA
CO
CT
DE
FL
GA
HI
ID
IL
IN
IA
KS
KY
LA
ME
MD
MA
MI
MN
MS
MO
MT
NE
NV
NH
NJ
NM
NY
NC
ND
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VT
VA
WA
WV
WI
WY


,State,Median Household income
0,Alabama,"$59,609"
1,Alaska,"$86,370"
2,Arizona,"$72,581"
3,Arkansas,"$56,335"
4,California,"$91,905"


In [41]:
merged_population_income = pd.merge(us_population_df,state_household_income_df)
merged_population_income.head()

,State,Population,id,Median Household income
0,Alabama,4903185,1,"$59,609"
1,Alaska,731545,2,"$86,370"
2,Arizona,7278717,4,"$72,581"
3,Arkansas,3017804,5,"$56,335"
4,California,39512223,6,"$91,905"


We decided it would be better to use population density to compare against median household income 

here we are extracting the area of us states by scraping a wikipedia page

In [24]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area'
htmlresponse = pd.read_html(wiki_url,header=0)

In [33]:
state_area_df.head()


,State / territory,Land area
0,State / territory,mi2
1,Alaska,570641
2,Texas,261232
3,California,155779
4,Montana,145546


In [36]:
state_area_df = htmlresponse[0]
state_area_df = state_area_df.drop(0)
state_area_df = state_area_df[['State / territory','Land area']]
state_area_df['Land area'] =pd.to_numeric(state_area_df['Land area']) 
state_area_df = state_area_df.rename(columns={'State / territory': 'State'})
state_area_df.head()


,State,Land area
1,Alaska,570641
2,Texas,261232
3,California,155779
4,Montana,145546
5,New Mexico,121298


Merging the two dataframes

In [42]:
merged_population_income = pd.merge(merged_population_income,state_area_df)
merged_population_income.head()

,State,Population,id,Median Household income,Land area
0,Alabama,4903185,1,"$59,609",50645
1,Alaska,731545,2,"$86,370",570641
2,Arizona,7278717,4,"$72,581",113594
3,Arkansas,3017804,5,"$56,335",52035
4,California,39512223,6,"$91,905",155779


creating population density colum   



In [43]:
merged_population_income["Population/SqMi"] = merged_population_income['Population'] / merged_population_income['Land area']
merged_population_income.head()

,State,Population,id,Median Household income,Land area,Population/SqMi
0,Alabama,4903185,1,"$59,609",50645,96.814789
1,Alaska,731545,2,"$86,370",570641,1.281971
2,Arizona,7278717,4,"$72,581",113594,64.076597
3,Arkansas,3017804,5,"$56,335",52035,57.995657
4,California,39512223,6,"$91,905",155779,253.642808


I knew new jersey was Us most populated states so I checked that my data reflected this before proceeding

In [44]:
from vega_datasets import data

states_geo = alt.topo_feature(data.us_10m.url, 'states')
merged_population_income["Population/SqMi"]=pd.to_numeric(merged_population_income["Population/SqMi"])

alt.Chart(states_geo).mark_geoshape().encode(
    color='Population/SqMi:Q',
    tooltip=['State:N', 'Population/SqMi:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(merged_population_income, 'id', ['Population/SqMi','State'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)



/Users/maddonk/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/maddonk/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/maddonk/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/maddonk/anaconda3/lib

alt.Chart(...)

creating integer value for median income

In [45]:
merged_population_income['Income_Integer'] = merged_population_income['Median Household income'].replace('[\$,]', '', regex=True).astype(int)
merged_population_income['Population/SqMi'] = pd.to_numeric(merged_population_income['Population/SqMi'], errors='coerce')
scatter = alt.Chart(merged_population_income).mark_circle().encode(
    y= alt.Y('Income_Integer',title="Median Household Income"),
    x='Population/SqMi',
    tooltip=['State:N', 'Income_Integer:Q', 'Population/SqMi:Q']
)
regressionline = scatter.transform_regression('Income_Integer', 'Population/SqMi').mark_line()
combined_chart = (scatter + regressionline)
combined_chart.interactive()


/Users/maddonk/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/maddonk/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [46]:
from  sqlalchemy import create_engine
engine = create_engine("sqlite:///PoliticsandDataSci.db")
merged_population_income.to_sql('US State Data',con=engine,if_exists='replace',index=False)


50

In [48]:
from sqlalchemy import create_engine, MetaData, Table

# Replace 'your_database_url' with the actual database URL
engine = create_engine('sqlite:///PoliticsandDataSci.db')

# Replace 'your_table_name' with the actual table name
table_name = 'US State Data'

metadata = MetaData(bind=engine)
table = Table(table_name, metadata, autoload=True, autoload_with=engine)

if table.exists():
    print(f"The table '{table_name}' exists.")
else:
    print(f"The table '{table_name}' does not exist.")

The table 'US State Data' exists.


/var/folders/rr/8gfsct5527d2w34_mknbmycm0000gn/T/ipykernel_99299/2226177565.py:12: SADeprecationWarning: The Table.exists() method is deprecated and will be removed in a future release.  Please refer to Inspector.has_table(). (deprecated since: 1.4)
  if table.exists():


In [49]:
from sqlalchemy import select

# Assuming 'table' is your Table object
with engine.connect() as connection:
    query = select([table]).limit(1)
    result = connection.execute(query).fetchall()

if result:
    print(f"The table '{table_name}' has data.")
else:
    print(f"The table '{table_name}' is empty.")

The table 'US State Data' has data.
